In [30]:
import os
import boto3
import re
import sagemaker
import numpy as np
import pandas as pd
import sklearn.model_selection
from sklearn.model_selection import train_test_split

# Import, subject to change once imported to aws
df = pd.read_csv("user_data.csv")

# Data Cleaning here extreme for now
df = df.drop(columns=['visitId', 'visitStartTime', 'visitNumber', 'date', 'totals', 'trafficSource', 'device', 'customDimensions'
                , 'socialEngagementType', 'totals', 'trafficSource', 'geoNetwork', 'Unnamed: 0'])
sites_id = pd.unique(df['site'])
for x in range(len(sites_id)):
    df['site'] = df['site'].replace([sites_id[x]], x)

times = []
for x in df.index:
    temp = str(df.time[x])
    temp = temp.split(':')
    secs = float(temp[1])
    secs += float(temp[2])
    times.append(secs)
times = pd.DataFrame(times)
df = df.assign(time=times)
# Splits data into train and test frames
train, test = train_test_split(df, test_size=.3)

# Finds unique users and pages
def get_unique_count(df):
    return len(pd.unique(df['fullVisitorId'])), len(pd.unique(df['site']))

# Stores number of users/pages for later usage
n_user, n_item = get_unique_count(train)
%store n_user
%store n_item
%store sites_id

# Making bucket for transport of data
session = boto3.session.Session()
region = session.region_name
sage_session = sagemaker.Session()
bucket = sage_session.default_bucket()

train_path = os.path.join("Recommendation Engine/data/", "train.npy")
test_path = os.path.join("Recommendation Engine/data/", "test.npy")

# Creates local dataset to represent test and train from code.
np.save('train', train.values)
np.save('test', test.values)

# Upload data to model.
sage_session.upload_data('test.npy', bucket=bucket, key_prefix='data')
sage_session.upload_data('train.npy', bucket=bucket, key_prefix='data')


Stored 'n_user' (int)
Stored 'n_item' (int)
Stored 'sites_id' (ndarray)


's3://sagemaker-us-east-1-350614259618/data/train.npy'

In [28]:
print(len(sites_id))

157
